# TAP Affect

This notebook was used as part of the [HETA project](http://heta.io) to experiment with [TAP](https://github.com/heta-io/tap) affect thresholds. It makes use of the [TapCliPy](https://github.com/heta-io/tapclipy) python client for TAP to call the `affectExpressions` query.

To use this notebook for your own tests, you will need:
    1. The URL of your TAP server
    2. Save the text files that you want to work with into the same directory as this notebook

In [208]:
# Install the TAP Python Client
!pip install 'tapclipy>=0.1.4'

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [209]:
# Import the client library
from tapclipy import tap_connect

### Connect to TAP and retrieve the current schema
As TAP exposes a GraphQL API, there can be changes in the schema over time. After connecting to TAP, this schema needs to be loaded into the client.

In [ ]:
# Set the url for your TAP server
tapURL = 'http://localhost:9000'
# Create TAP Connection
tap = tap_connect.Connect(tapURL)
# Load the Current Schema
tap.fetch_schema()

In [212]:
#Print out schema fields
for query,type in tap.schema_query_name_types().items():
    print("{} >> {}".format(query, type))

clean >> StringResult
annotations >> SentencesResult
vocabulary >> VocabResult
metrics >> MetricsResult
posStats >> PosStatsResult
syllables >> SyllablesResult
spelling >> SpellingResult
expressions >> ExpressionsResult
reflectExpressions >> ReflectExpressionsResult
affectExpressions >> AffectExpressionsResult
moves >> StringListResult


### Setup Query
The client includes built in queries. We can either use the client query for `affectExpressions` or we can create our own. If creating from scratch, it wise to use the client query as a template to ensure the query is properly formed.

In [213]:
# Get query from client
query = tap.query('affectExpressions')
print(query)


query AffectExpressions($input:String,$parameters:String) { 
    affectExpressions(text:$input,parameters:$parameters) { 
        querytime
        message
        timestamp
        analytics {
            affect {
                text
                valence
                arousal
                dominance
                startIdx
            }
        }
    }}



### Helper functions

To make it easier to run repeated tests on different files, we can setup some helper functions.

In [214]:
# Open a text file and return it as a string
def readFile(filename):
    file = open(filename)
    text = file.read()
    file.close()
    return text

In [218]:
# Test on a file
myText = readFile('dummy-affect.txt')
myText

'I am extremely happy to present this amazing text file for analysis. I hope it performs to expectations.'

In [225]:
# Get Affect Analytics from TAP and format the results
def textAffect(text,arousal=0.0,valence=0.0,dominance=0.0):
    parameters = '{"valence":'+str(valence)+',"arousal":'+str(arousal)+',"dominance":'+str(dominance)+'}'
    json = tap.analyse_text(query, text,parameters)
    analytics = json['data']['affectExpressions']['analytics']
    filtered = [x['affect'] for x in analytics if x['affect']]
    flattened = [item for sublist in filtered for item in sublist]
    #print(flattened)
    numFiltered = len(flattened)
    numLexicon = len(analytics)
    words = text.split(' ')
    numWords = len(words)
    percentAffect = numFiltered/numWords*100
    print("{0} words matched out of {1} total words in the text - {2} percent".format(numFiltered,numWords,percentAffect))
    for t in flattened:
        #print(t)
        print(t['text'],'\t[a] ',t['arousal'],' [v] ',t['valence'],' [d] ',t['dominance'])
    

In [226]:
# Test the function on our text
textAffect(myText,arousal=4.95)

4 words matched out of 18 total words in the text - 22.22222222222222 percent
happy 	[a]  6.05  [v]  8.47  [d]  7.21
amazing 	[a]  5  [v]  7.24  [d]  5.83
hope 	[a]  5.29  [v]  7.48  [d]  6.78
performs 	[a]  5.15  [v]  6.48  [d]  5.82


### Do analysis

Load the file, then check the results with different values of `arousal`, `valence`, and `dominance`.

In [227]:
# Read file
text1 = readFile('dummy-affect.txt')
# Check values
textAffect(text1,arousal=4.0,valence=5.0,dominance=0.0)

6 words matched out of 18 total words in the text - 33.33333333333333 percent
happy 	[a]  6.05  [v]  8.47  [d]  7.21
present 	[a]  4.82  [v]  6.85  [d]  6.65
amazing 	[a]  5  [v]  7.24  [d]  5.83
hope 	[a]  5.29  [v]  7.48  [d]  6.78
performs 	[a]  5.15  [v]  6.48  [d]  5.82
expectations 	[a]  4.5  [v]  6.1  [d]  5.05


In [228]:
# Check different values
textAffect(text1,arousal=5.0,valence=7.0,dominance=5.0)

3 words matched out of 18 total words in the text - 16.666666666666664 percent
happy 	[a]  6.05  [v]  8.47  [d]  7.21
amazing 	[a]  5  [v]  7.24  [d]  5.83
hope 	[a]  5.29  [v]  7.48  [d]  6.78
